In [1]:
import torch.nn as nn
import torch
#import torchvision
import numpy as np

In [2]:
#自定义卷积操作
def corr2d(X, K):
    h, w = K.shape#卷积核的尺寸
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))#确保卷积核滑动操作不会越界
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()#卷积操作
    return Y

In [3]:
class Conv2D(nn.Module):#继承了nn.Module，定义每个卷积层
    def __init__(self, kernel_size):#初始化的时候需要提供卷积核尺寸
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))
        #nn.Parameter将一个固定不可训练的tensor转换成可以训练的类型parameter，
        # 并将这个parameter绑定到这个module里面(net.parameter()中就有这个绑定的parameter，
        # 所以在参数优化的时候可以进行优化的)，所以经过类型转换这个self.weight变成了模型的一部分，
        # 成为了模型中根据训练可以改动的参数了

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias#模型的正向推导

In [4]:
kernel_size = (2,2)#设置卷积核大小
x =torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])#模型的输入量
conv2d = Conv2D(kernel_size)#经过一次的卷积层处理
print("显示卷积层处理后的输出\n",conv2d(x))#显示卷积层处理后的输出，为一个tensor

显示卷积层处理后的输出
 tensor([[5.7277, 5.0590],
        [3.7218, 3.0531]], grad_fn=<AddBackward0>)


In [5]:
# 定义一个函数来计算卷积层。它对输入和输出做相应的升维和降维
def comp_conv2d(conv2d, X):
    # (1, 1)代表批量大小和通道数均为1
    #原X尺度为（８，８），经过x.view后，尺度为（１，１，８，８）
    X = X.view((1, 1) + X.shape)
    Y = conv2d(X)
    return Y.view(Y.shape[2:])  # 排除不关心的前两维:批量和通道
# 注意这里是两侧分别填充1行或列，所以在两侧一共填充2行或列
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)
#padding控制zero-padding的数目
X = torch.rand(8, 8)
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [6]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape#计算正常情况下卷积层操作后的尺度

torch.Size([4, 4])

In [7]:
def pool2d(X, pool_size, mode='max'):#pool_size为池化层大小
    X = X.float()#x值进行浮点型转换
    p_h, p_w = pool_size#pool_size为池化层大小
    Y = torch.zeros(X.shape[0] - p_h + 1, X.shape[1] - p_w + 1)#池化操作后的图像存储的大小，初始时刻都是设置为０
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()#取极大值
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()#取均值
    return Y

In [8]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
print("极大值池化\n",pool2d(X, (2, 2)))#输出显示X经过极大值池化后的输出结果

极大值池化
 tensor([[4., 5.],
        [7., 8.]])


In [9]:
print("均值池化\n",pool2d(X, (2, 2), 'avg'))#输出显示X经过均值池化后的输出结果

均值池化
 tensor([[2., 3.],
        [5., 6.]])


In [10]:
X = torch.arange(16, dtype=torch.float).view((1, 1, 4, 4))
pool2d = nn.MaxPool2d(3, padding=1, stride=2)#采用torch中的最大池化函数实现
print("采用nn.MaxPool2d获得的最大池化结果\n",pool2d(X))

采用nn.MaxPool2d获得的最大池化结果
 tensor([[[[ 5.,  7.],
          [13., 15.]]]])


In [11]:
import torch.nn as nn
import torch

class SelfConv(nn.Module):#构建卷积网络
    def __init__(self):
        super(SelfConv, self).__init__()
        #该网络中采用了一个卷积层，一个激活函数与池化层
        self.conv = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=(3,3),stride=1,padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))

    def forward(self,x):
        x = self.conv(x)#输入的tensor先进行卷积
        print("conv.shape:{}".format(x.shape))
        x = self.relu(x)
        x = self.pool(x)
        print("pool.shape:{}".format(x.shape))

        return x

input = torch.randn(size=(1,3,224,224),dtype=torch.float32)
model = SelfConv()#自定义模型实例化
output = model(input)#通过自定义模型对输入变量进行运算，获得输出

conv.shape:torch.Size([1, 32, 224, 224])
pool.shape:torch.Size([1, 32, 112, 112])
